In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, StringVar, OptionMenu, Label, Button
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class DataComparisonApp:
    def __init__(self, master, include_compare_button=True):
        self.master = master
        self.master.config(background="white")
        self.master.title("Veri İşlemleri")

        # Dosya seçme arayüzü için etiket ve butonlar
        self.file_label = tk.Label(master, text="CSV Dosyasını Seçiniz", background="white")
        self.file_label.place(x=850, y=5)

        self.file_path_var = tk.StringVar()
        self.file_entry = tk.Label(master, textvariable=self.file_path_var, background="white")
        self.file_entry.place(x=10, y=30)

        self.load_file_button = tk.Button(master, text="Dosya Seç", command=self.load_file, background="white")
        self.load_file_button.place(x=880, y=30)

        # Boş alanı yerleştir
        self.create_empty_label()

        # Diğer işlemler için butonlar
        self.normalize_button = tk.Button(master, text="Normalize", command=self.normalize_data, background="white")
        self.normalize_button.place(x=25, y=520)

        self.categorize_button = tk.Button(master, text="Kategorize", command=self.categorize_data, background="white")
        self.categorize_button.place(x=25, y=480)
        
        self.column1_label = Label(master, text="Karşılaştırılacak Birinci Sütun:", background="white")
        self.column1_label.place(x=25, y= 250)

        self.column1_var = StringVar()
        self.column1_menu = OptionMenu(master, self.column1_var, "")  # Boş bir menü ile başlayalım
        self.column1_menu.place(x=25, y= 280)

        self.column2_label = Label(master, text="Karşılaştırılacak İkinci Sütun:", background="white")
        self.column2_label.place(x=25, y= 320)

        self.column2_var = StringVar()
        self.column2_menu = OptionMenu(master, self.column2_var, "")
        self.column2_menu.place(x=25, y= 350)
        
        self.baslangic_yil_var = Label(master, text="Başlangıç yılı:", background="white")
        self.baslangic_yil_var.place(x=25, y= 100)

        # Başlangıç yılı seçimi için combobox
        self.start_year_combobox = ttk.Combobox(master, state="readonly", background="white")
        self.start_year_combobox.place(x=25, y= 130)
        
        self.bitis_yil_var = Label(master, text="Bitiş yılı:", background="white")
        self.bitis_yil_var.place(x=25, y= 160)

        # Bitiş yılı seçimi için combobox
        self.end_year_combobox = ttk.Combobox(master, state="readonly", background="white")
        self.end_year_combobox.place(x=25, y= 190)
        
        self.compare_button = tk.Button(master, text="Karşılaştır", command=self.compare_columns, background="white")
        self.compare_button.place(x=25, y=395)
        

        self.histogram_button = tk.Button(master, text="Histogram Çiz", command=self.plot_histogram, background="white")
        self.histogram_button.place(x=25, y= 560)

        self.clear_button = tk.Button(master, text="Temizle", command=self.clear_all, background="white")
        self.clear_button.place(x=885, y=610)

        # Sonuç etiketleri
        self.result_label_normalize = tk.Label(master, text="", background="white")
        self.result_label_normalize.place(x=400, y=250)

        self.result_label_categorize = tk.Label(master, text="", background="white")
        self.result_label_categorize.place(x=400, y=250)

        self.result_label_histogram = tk.Label(master, text="", background="white")
        # self.result_label_histogram.place(x=0, y=100)

        self.result_label_compare = tk.Label(master, text="", background="white")
        self.result_label_compare.place(x=300, y=200)

        # Karşılaştırma grafiği canvas'ını saklamak için
        self.canvas_compare = None
        # Histogram canvas'ını saklamak için
        self.canvas_histogram = None

    def create_empty_label(self):
        empty_var = tk.StringVar()
        empty_label = tk.Label(self.master, textvariable=empty_var, background="white")
        # Boş alanı yerleştir
        empty_label.place(x=10, y=250, width=400, height=300)

    def load_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            self.file_path_var.set(file_path)
            df = pd.read_csv(file_path)
            years = df['Year'].unique()
            self.start_year_combobox['values'] = tuple(years)
            self.end_year_combobox['values'] = tuple(years)
            self.start_year_combobox.set("Select Start Year")
            self.end_year_combobox.set("Select End Year")

            # Veri setindeki sütun isimlerini al
            column_options = [col for col in df.columns[2:]]

            # İlk seçeneği belirle
            self.column1_menu['menu'].delete(0, 'end')
            for col in column_options:
                self.column1_menu['menu'].add_command(label=col, command=lambda x=col: self.column1_var.set(x))
            self.column1_var.set(column_options[0])

            # İkinci seçeneği belirle
            self.column2_menu['menu'].delete(0, 'end')
            for col in column_options:
                self.column2_menu['menu'].add_command(label=col, command=lambda x=col: self.column2_var.set(x))
            self.column2_var.set(column_options[1])

    def normalize_data(self):
        file_path = self.file_path_var.get()
        if file_path:
            df = pd.read_csv(file_path)
            normalized_df = self.normalize(df, ['Price_beef_kilo', 'Price_coffee_kilo', 'Price_rice_kilo'])
            self.show_normalized_data(normalized_df)
        else:
            print("Lütfen bir CSV dosyası seçin.")

    def normalize(self, data, columns):
        normalized_data = data.copy()
        for column in columns:
            normalized_data[column] = data[column] / data[column].max()

        return normalized_data[['Price_beef_kilo', 'Price_coffee_kilo', 'Price_rice_kilo']]

    def show_normalized_data(self, normalized_df):
        # Normalleştirilmiş verileri göstermeden önce eski grafikleri temizle
        self.clear_all()
        result_text = "Normalleştirilmiş Veriler:\n" + str(normalized_df)
        self.result_label_normalize.config(text=result_text)

    def categorize_data(self):
        file_path = self.file_path_var.get()
        if file_path:
            df = pd.read_csv(file_path)
            result_beef, result_rice, result_coffee = self.categorize(df)
            self.show_categorized_data("Kategorize Edilmiş Veriler - Beef", result_beef)
            self.show_categorized_data("Kategorize Edilmiş Veriler - Rice", result_rice)
            self.show_categorized_data("Kategorize Edilmiş Veriler - Coffee", result_coffee)
        else:
            print("Lütfen bir CSV dosyası seçin.")

    def categorize(self, data):
        bins_beef = np.linspace(data['Price_beef_kilo'].min(), data['Price_beef_kilo'].max(), 4)
        categories_beef = ['low', 'mid', 'high']
        data['price-category_beef'] = pd.cut(data['Price_beef_kilo'], bins_beef, labels=categories_beef, include_lowest=True)
        result_beef = data[['Price_beef_kilo', 'price-category_beef']].head(350)

        bins_rice = np.linspace(data['Price_rice_kilo'].min(), data['Price_rice_kilo'].max(), 4)
        categories_rice = ['low', 'mid', 'high']
        data['price-category_rice'] = pd.cut(data['Price_rice_kilo'], bins_rice, labels=categories_rice, include_lowest=True)
        result_rice = data[['Price_rice_kilo', 'price-category_rice']].head(350)

        bins_coffee = np.linspace(data['Price_coffee_kilo'].min(), data['Price_coffee_kilo'].max(), 4)
        categories_coffee = ['low', 'mid', 'high']
        data['price-category_coffee'] = pd.cut(data['Price_coffee_kilo'], bins_coffee, labels=categories_coffee, include_lowest=True)
        result_coffee = data[['Price_coffee_kilo', 'price-category_coffee']].head(350)

        return result_beef, result_rice, result_coffee

    def show_categorized_data(self, title, dataframe):
        # Kategorize edilmiş verileri göstermeden önce eski grafikleri temizle
        self.clear_all()
        result_text = title + ":\n" + str(dataframe)
        self.result_label_categorize.config(text=result_text)

    def compare_columns(self):
        file_path = self.file_path_var.get()
        column1 = self.column1_var.get()
        column2 = self.column2_var.get()
        start_year = self.start_year_combobox.get()
        end_year = self.end_year_combobox.get()

        if file_path and column1 and column2 and start_year and end_year:
            df = pd.read_csv(file_path)
            df_filtered = df[(df['Year'] >= int(start_year)) & (df['Year'] <= int(end_year))]
            self.compare_columns_by_year(df_filtered, column1, column2)
        else:
            print("Lütfen CSV dosyası, sütunlar ve yılları seçin.")

    def compare_columns_by_year(self, df, column1, column2):
        df = df.copy()
        df.loc[:, 'Date'] = df['Year'].astype(str) + '-' + df['Month']
        df.loc[:, 'Date'] = pd.to_datetime(df['Date'], format='%Y-%b')

        df.set_index('Date', inplace=True)

        df_selected = df[['Year', column1, column2]]
        df_grouped = df_selected.groupby(df.index.year).mean()

        # Önceki çizimleri temizle
        self.clear_graph_compare()

        # Yeni bir çizim yap
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.plot(df_grouped.index, df_grouped[column1], label=column1)
        ax.plot(df_grouped.index, df_grouped[column2], label=column2)
        ax.set_xlabel('Yıl')
        ax.set_ylabel('Değer')
        ax.legend()

        # Canvas'ı güncelle
        if not self.canvas_compare:
            self.canvas_compare = FigureCanvasTkAgg(fig, master=self.master)
            self.canvas_compare.get_tk_widget().place(x=250, y=80, width=600, height=550)
        else:
            self.canvas_compare.figure = fig
            self.canvas_compare.draw_idle()

        # Güncellenen çizimi göster
        self.canvas_compare.draw()

        # Sonuç etiketini temizle
        self.clear_result_label_compare()

    def plot_histogram(self):
        file_path = self.file_path_var.get()
        if file_path:
            df = pd.read_csv(file_path)
            # Histogram çizilmeden önce eski grafikleri temizle
            self.clear_all()
            self.show_histogram(df)
        else:
            print("Lütfen bir CSV dosyası seçin.")

    def show_histogram(self, data):
        # Histogram çizilmeden önce eski grafikleri temizle
        self.clear_all()

        fig, ax = plt.subplots(figsize=(8, 6))
        data.hist(bins=50, xlabelsize=8, ylabelsize=8, ax=ax)

        self.canvas_histogram = FigureCanvasTkAgg(fig, master=self.master)
        self.canvas_histogram.draw()
        self.canvas_histogram.get_tk_widget().place(x=250, y=80, width=600, height=550)

    def clear_all(self):
        # Tüm grafikleri ve sonuç etiketlerini temizle
        self.clear_graph_compare()
        self.clear_graph_histogram()
        self.clear_result_label_normalize()
        self.clear_result_label_categorize()
        self.clear_result_label_histogram()
        self.clear_result_label_compare()

    def clear_graph_compare(self):
        # Karşılaştırma grafiği canvas'ını temizle
        if self.canvas_compare:
            self.canvas_compare.get_tk_widget().destroy()
            self.canvas_compare = None

    def clear_graph_histogram(self):
        # Histogram canvas'ını temizle
        if self.canvas_histogram:
            self.canvas_histogram.get_tk_widget().destroy()
            self.canvas_histogram = None

    def clear_result_label_normalize(self):
        # Normalize sonuç etiketini temizle
        self.result_label_normalize.config(text="")

    def clear_result_label_categorize(self):
        # Kategorize sonuç etiketini temizle
        self.result_label_categorize.config(text="")

    def clear_result_label_histogram(self):
        # Histogram sonuç etiketini temizle
        self.result_label_histogram.config(text="")

    def clear_result_label_compare(self):
        # Karşılaştırma sonuç etiketini temizle
        self.result_label_compare.config(text="")

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Veri İşlemleri Uygulaması")

    data_comparison_app = DataComparisonApp(root, include_compare_button=True)

    # Ekran çözünürlüğünüze uygun bir değeri kullanabilirsiniz
    root.geometry("1000x680")
    root.mainloop()
